# 처음 배우는 셸 스크립트#5

## 6장 찾은 문자열을 바꿀 수 있는 sed

리눅스에서 sed를 이용하면 쉽게 변경하고자 하는 문자열을 찾아 변경할 수 있다  

### 6.1 sed 사용법
sed는 스트림 편집기이다. 스트림 편집기는 입력 스트림에서 텍스트 변환을 수행하는 데 사용된다.  
입력 스트림: 대상 파일  
스크립트: 어떤 텍스트를 어떻게 변환할지 정의. 대상 파일의 범위를 지정하는 어드레스+ 명령어  

가장 기본적인 사용법은  
sed 스크립트 대상파일
sed [옵션] '[어드레스] 명령어' 대상파일   
sed [옵션] -f 스크립트파일 대상파일
명령어 | sed [옵션] '스크립트'

대상파일이 변경되지 않고, 변경시의 결과물이 표준출력으로 나온다

In [13]:
%%bash
#!/bin/bash
## 명령어 s/찾을문자열/변경문자열/ 의 형태
cat /etc/ssh/sshd_config | grep '^#PermitRoot' -n
sed 's/#PermitRoot/PermitRoot/' /etc/ssh/sshd_config | grep '^PermitRoot' -n

34:#PermitRootLogin prohibit-password
34:PermitRootLogin prohibit-password


In [15]:
%%bash
#!/bin/bash
## 스크립트 파일
echo 's/#PermitRoot/PermitRoot/' > sed-script5.txt
sed -f sed-script5.txt /etc/ssh/sshd_config | grep '^PermitRoot' -n

34:PermitRootLogin prohibit-password


In [19]:
%%bash
#!/bin/bash
## 파이프연산자
cat /etc/ssh/sshd_config | sed -e 's/#PermitRoot/PermitRoot/' | grep '^PermitRoot'

PermitRootLogin prohibit-password


### 6.2 sed 스크립트
sed 스크립트는 어드레스와 명령어로 구성된다.  

어드레스  
대상 파일에서 범위를 정의한다.
|옵션|설명|
|----|---|
|number|명시된 숫자에 해당하는 라인 번호일 경우 명령어를 수행함|
|number~step|number부터 step간격으로 명령어를 수행함|
|$|파일의 마지막 라인일 경우 명령어를 수행함|
|/regexp/|명시한 정규 표현식과 일치하는 라인일 경우 명령어를 수행함|
|\cregexpc|명시한 정규 표현식과 일치하는 라인일 경우 다음 명령어를 수행함|
|0,addr2|1번째 라인부터 addr2가 포함된 라인까지가 범위이며, addr2는 정규표현식이어야 함|
|addr1,+N|addr1이 포함된 라인부터 +N라인 까지가 범위다. addr1은 정규표현식이고, N은 숫자여야 함|
|addr1,~N|addr1이 포함된 라인을 기준으로 N라인 까지가 범위다. addr1은 정규표현식이고, N은 숫자여야 함|

라인 넘버는 0부터 시작한다
명령어는 한 라인에 하나만 사용할 수 있다. 아니면 ;로 구분 가능하다.


In [20]:
%%bash
#!/bin/bash
## 예제 파일 확인. sed는 주로 환경 설정 파일의 특정 문자열을 변경할 때 사용된다
cat hosts

# This is Sed Sample File
# We will test to replace from a-text to b-text.
# It was created by NaleeJang.

127.0.0.1   localhost

# Development
192.168.100.250 git.example.com
192.168.100.10  servera.example.com
192.168.100.11  dev.example.com

# Test

172.10.2.12 test1.example.com
172.10.2.13 test2.example.com

# Production
122.10.10.31 service.example.com
122.10.10.32 service1.example.com
122.10.10.33 service2.example.com


In [24]:
%%bash
#!/bin/bash
## -n 옵션은 대상 파일 내용을 출력하지 않겠다는 옵션
## '5 p'는 어드레스 5, 명령어 p(print)
sed -n '5 p' hosts

127.0.0.1   localhost


In [38]:
%%bash
#!/bin/bash
## = 명령어는 라인의 라인 번호를 출력하라는 의미
sed -n '1~3 = ' hosts

1
4
7
10
13
16
19


In [41]:
%%bash
#!/bin/bash
## $ 어드레스는 파일의 마지막 라인
sed -n '$ =' hosts
sed -n '$ p' hosts

20
122.10.10.33 service2.example.com


In [43]:
%%bash
#!/bin/bash
## /regexp/
sed -n '/test[0-9].*/ p' hosts
sed -n '\ctest[0-9].*c p' hosts


172.10.2.12 test1.example.com
172.10.2.13 test2.example.com
172.10.2.12 test1.example.com
172.10.2.13 test2.example.com


In [55]:
%%bash
#!/bin/bash
## 0부터  # Devel로 시작하는 라인까지 출력
#sed -n '0,/^# Devel*/ p' hosts
## # Devel로 시작하는 라인부터 아래 3줄까지 출력
sed -n '/^# Devel*/,+3 p' hosts
## # Devel로 시작하는 라인을 포함해 3줄까지 출력
sed -n '/^# Devel*/,~3 p' hosts


# Development
192.168.100.250 git.example.com
192.168.100.10  servera.example.com
192.168.100.11  dev.example.com
# Development
192.168.100.250 git.example.com
192.168.100.10  servera.example.com


명령어  

0 OR 1 어드레스 명령어  
0 OR 1 어드레스 명령어는 라벨, 주석, 블록과 같이 파일 내용에 아무런 영향을 주지 않는 명령어와  
문자열 추가, 삽입, 스크립트 종료, 파일 내용 추가와 같은 명령어로 어드레스가 필요한 명령어가 있다.
|옵션|설명|
|---|----|
|:label|라벨|
|#comment|주석|
|{ ... }|블록|
|=|현재 라인 번호 출력|
|a \ text| 문자열 추가|
|i \ text| 문자열 삽입|
|q|스크립트 실행 종료|
|Q|스크립트 실행 종료|
|r 파일명|파일 내용 추가|
|R 파일명|파일 첫라인 추가|

In [47]:
%%bash
#!/bin/bash
cat sed-script.txt
sed -n -f sed-script.txt hosts

/# Test/,+3 {
=
# first label
:label1
}
12
13
14
15


In [5]:
%%bash
#!/bin/bash
sed -n '/172.10.2.13/ { a\172.10.2.14 test3.example.com
p}' hosts

172.10.2.13 test2.example.com
172.10.2.14 test3.example.com


In [104]:
%%bash
#!/bin/bash
sed -n '/172.10.2.13/ { i\172.10.2.14 test3.example.com 
p}' hosts

172.10.2.14 test3.example.com 
172.10.2.13 test2.example.com


In [7]:
%%bash
#!/bin/bash
#q는 수행중이던 스크립트를 종료한다. 따라서  텍스트 추가가 종료되고 그냥 추가될 텍스트를 p한다.
sed -n '/172.10.2.13/ {a\172.10.2.14 test3.example.com 
q
p}' hosts

172.10.2.14 test3.example.com 


In [8]:
%%bash
#!/bin/bash
#Q는 수행중이던 스크립트를 종료한다. 뒤에 출력도 안한다
sed -n '/172.10.2.13/ {a\172.10.2.14 test3.example.com 
Q
p}' hosts

In [9]:
%%bash
#!/bin/bash
cat sed-read.txt
echo ""
sed -n '/172.10.2.13/ {r sed-read.txt
p}' hosts

172.10.2.14 test3.example.com
172.10.2.15 test4.example.com

172.10.2.13 test2.example.com
172.10.2.14 test3.example.com
172.10.2.15 test4.example.com


In [10]:
%%bash
#!/bin/bash
##R은 해당 파일의 첫 라인만 읽어서 추가한다
sed -n '/172.10.2.13/ {R sed-read.txt
p}' hosts

# This is Sed Sample File
# We will test to replace from a-text to b-text.
# It was created by NaleeJang.

127.0.0.1   localhost

# Development
192.168.100.250 git.example.com
192.168.100.10  servera.example.com
192.168.100.11  dev.example.com

# Test

172.10.2.12 test1.example.com
172.10.2.13 test2.example.com
172.10.2.14 test3.example.com

# Production
122.10.10.31 service.example.com
122.10.10.32 service1.example.com
122.10.10.33 service2.example.com


어드레스 범위 명령어  
범위도 받을 수 있다  
|옵션|설명|
|----|----|
|b, label|라벨을 호출함|
|c\text|앞에서 명시된 패턴이 포함된 라인을 text로 변경|
|d, B|앞에서 명시된 패턴 삭제|
|h, H|패턴 공간을 홀드 공간에 복사/추가|
|g, G|홀드 공간을 패턴 공간에 복사/추가|
|l|입력된 데이터의 현재 라인 출력|
|l width| 명시한 너비에 맞게 입력된 데이터의 현재 라인 출력|
|n N|입력된 데이터의 다음 라인을 복사/추가|
|p P|현재 패턴 공간 출력|
|s/regexp/replacement/| 정규표현식에 해당하는 데이터를 replacement로 변경함|
|t label / T label| 앞에서 선언된 명령어를 실행 후 라벨로 분기|
|w 파일명 / W 파일명| 명시한 파일에 현재 패턴 공간을 저장|
|x|홀드와 패턴 공간의 콘텐츠를 교환함|
|y/source/dest/|패턴이 포함된 라인의 문자열을 dest문자열로 변경|

In [13]:
%%bash
#!/bin/bash
## b label 로 goto
cat sed-script1.txt
echo ""
sed -n -f sed-script1.txt hosts

/# Test/,+3 {
# if input line is empty, doesn’t execute replacing
/^$/ b label1
s/[tT]est/dev/
: label1
p
}

# dev

172.10.2.12 dev1.example.com
172.10.2.13 dev2.example.com


In [16]:
%%bash
#!/bin/bash
## c\text로 변경
sed '/service.e/ c\122.10.10.30 vip.service.example.com' hosts | tail -n4

# Production
122.10.10.30 vip.service.example.com
122.10.10.32 service1.example.com
122.10.10.33 service2.example.com


In [19]:
%%bash
#!/bin/bash
## d, D로 삭제. d는 뉴라인과 상관없이 해당 문자열이 포함된 라인을 삭제, D는 뉴라인까지만 삭제

sed -n '0,/NaleeJang/ {
s/We will test to replace/We will test to\nreplace/
/test/ d
p }' hosts
echo ""
sed -n '0,/NaleeJang/ {
s/We will test to replace/We will test to\nreplace/
/test/ D
p }' hosts

# This is Sed Sample File
# It was created by NaleeJang.

# This is Sed Sample File
replace from a-text to b-text.
# It was created by NaleeJang.


In [28]:
%%bash
#!/bin/bash
## h, H는 홀드 버퍼의 내용을 패턴 버퍼로 복사한다
## p는 패턴 버퍼의 내용을 출력한다
## 잘못된 예제
sed -n '/Product/,+3 {
s/service/Production/
h
p}' hosts

# Production
122.10.10.31 Production.example.com
122.10.10.32 Production1.example.com
122.10.10.33 Production2.example.com


패턴 버퍼와 홀드 버퍼  
sed는 명령어를 수행할 때 대상 파일의 내용을 한 줄씩 읽어 패턴 버퍼(패턴 공간)에 삽입한다  
패턴 버퍼는 현재 정보를 저장하는 버퍼 메모리이며, sed를 통해 출력을 실행하면 패턴 버퍼의 내용이 인쇄된다

이외에도 홀드 버퍼(홀드 공간)이라 불리는 임시 버퍼 메모리 공간이 있다.  
홀드 버퍼에는 대상 파일의 라인 수만큼의 빈 공간을 가지고 있으며, sed가 다른 라인을 처리할 때 재사용한다  
대부분의 명령어는 패턴 버퍼에서 작업이 이루어진다 


In [35]:
%%bash
#!/bin/bash
## g, G는 패턴 버퍼의 내용을 홀드 버퍼로 복사한다
## 대문자 H,G를 사용하면 변경과정이 나옴
sed -n '/Product/,+3 {
s/service/Production/
H
s/122.10.10/199.9.9/
G
p}' hosts

# Production

# Production
199.9.9.31 Production.example.com

# Production
122.10.10.31 Production.example.com
199.9.9.32 Production1.example.com

# Production
122.10.10.31 Production.example.com
122.10.10.32 Production1.example.com
199.9.9.33 Production2.example.com

# Production
122.10.10.31 Production.example.com
122.10.10.32 Production1.example.com
122.10.10.33 Production2.example.com


## 6.3 sed 옵션
|옵션|설명|
|-n,--quiet,--silent|현재 패턴 공간을 출력하지 않음|
|-e 스크립트, --expression=스크립트|여러 개의 스크립트를 실행할 때 사용함|
|-f 스크립트파일, --file=스크립트파일|파일로 실행|
|--folow-symlinks|심볼릭 링크 대상파일에 사용|
|-i파일 확장자, --in-place=파일확장자|현재 파일에 결과를 그대로 저장함. 파일확장자를 넣으면 백업 파일이 생성됨|
|-c, --copy|파일명 뒤에 c가 붙은 백업파일이 생성됨|
|-r,--regexp-extended|sed는 기본적으로 basic regex이다. 확장으로 변경함|
|-s, --separate|여러 개의 파일을 각각 처리한다|
|-u, --unbuffered|버퍼를 자주 비워 성능이 향상됨|
|-z,--null-data|구분 기호가 null인 데이터의 문자열을 변경할 때 사용|
-E is for compatibility with BSD. It's the same as -r or --regexp-extended -


https://mug896.github.io/sed-stream-editor/index.html 를 참고하자
